In [14]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
prathamjain965_aridzip_path = kagglehub.dataset_download('prathamjain965/aridzip')

print('Data source import complete.')


Data source import complete.


In [15]:
print(prathamjain965_aridzip_path)

/kaggle/input/aridzip


In [1]:
!pip install opencv-python
!pip install scikit-learn
!pip install matplotlib
!pip install tensorboard
!pip install fvcore
!pip install simplejson
!pip install av
!pip install pytorchvideo

  Using cached av-14.4.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.6 kB)
Using cached av-14.4.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (35.3 MB)
  Using cached pytorchvideo-0.1.5.tar.gz (132 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188686 sha256=70265cab032b85d67b2651dcd898ec5eca14396f2926d79fa5008119a15464f5
  Stored in directory: /root/.cache/pip/wheels/a4/6d/ae/d016375a73be141a0e11bb42289e2d0b046c35687fc8010ecc
Successfully built pytorchvideo


In [17]:
!git clone https://github.com/facebookresearch/slowfast

fatal: destination path 'slowfast' already exists and is not an empty directory.


In [3]:
%cd SlowFast

/content/SlowFast


In [4]:
!python setup.py build develop

running build
running build_py
creating build/lib/slowfast
copying slowfast/__init__.py -> build/lib/slowfast
creating build/lib/slowfast/config
copying slowfast/config/defaults.py -> build/lib/slowfast/config
copying slowfast/config/__init__.py -> build/lib/slowfast/config
copying slowfast/config/custom_config.py -> build/lib/slowfast/config
creating build/lib/slowfast/utils
copying slowfast/utils/parser.py -> build/lib/slowfast/utils
copying slowfast/utils/ava_eval_helper.py -> build/lib/slowfast/utils
copying slowfast/utils/metrics.py -> build/lib/slowfast/utils
copying slowfast/utils/multiprocessing.py -> build/lib/slowfast/utils
copying slowfast/utils/__init__.py -> build/lib/slowfast/utils
copying slowfast/utils/meters.py -> build/lib/slowfast/utils
copying slowfast/utils/misc.py -> build/lib/slowfast/utils
copying slowfast/utils/lr_policy.py -> build/lib/slowfast/utils
copying slowfast/utils/multigrid.py -> build/lib/slowfast/utils
copying slowfast/utils/env.py -> build/lib/slow

In [5]:
!python -c "import slowfast; print('PySlowFast installed successfully')"

PySlowFast installed successfully


In [7]:
!mkdir -p /content/pretrained_models

In [8]:
import torch
import torchvision

In [10]:
model_url = "https://dl.fbaipublicfiles.com/pyslowfast/model_zoo/mvitv2/pysf_video_models/MViTv2_S_16x4_k400_f302660347.pyth"

In [11]:
!wget -O /content/pretrained_models/mvit_v2_s_kinetics400.pth {model_url}

--2025-06-26 09:45:37--  https://dl.fbaipublicfiles.com/pyslowfast/model_zoo/mvitv2/pysf_video_models/MViTv2_S_16x4_k400_f302660347.pyth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.15, 13.226.210.25, 13.226.210.111, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 414835762 (396M) [binary/octet-stream]
Saving to: ‘/content/pretrained_models/mvit_v2_s_kinetics400.pth’

/content/pretrained 100%[===================>] 395.62M  60.7MB/s    in 6.5s    

2025-06-26 09:45:44 (61.3 MB/s) - ‘/content/pretrained_models/mvit_v2_s_kinetics400.pth’ saved [414835762/414835762]



In [12]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [28]:
def parse_arid_splits(arid_path):
    """
    Parse your ARID dataset structure for evaluation
    """
    # ARID v1.5 action categories
    action_categories = [
        'Drink', 'Jump', 'Pick', 'Pour', 'Push',
        'Run', 'Sit', 'Stand', 'Turn', 'Walk', 'Wave'
    ]

    clips_path = os.path.join(arid_path, "clips_v1.5")
    list_path = os.path.join(arid_path, "list_cvt/split_0")

    # Parse test split file
    test_data = []
    test_file = os.path.join(list_path, "split0_test.txt")

    with open(test_file, 'r') as f:
        lines = f.readlines()

    for line in lines:
        line = line.strip()
        if line:
            parts = line.split()
            if len(parts) >= 2:
                video_path = parts[2]  # e.g., "action_folder/video_name.mp4"
                label = int(parts[1])
                # Full path to video file
                full_video_path = os.path.join(clips_path, video_path)
                if os.path.exists(full_video_path):
                    test_data.append({
                        'video_path': video_path,  # Adjust path format
                        'full_path': full_video_path,
                        'label': label,
                        'action': action_categories[label]
                    })

    return pd.DataFrame(test_data), action_categories

# Set your ARID dataset path
ARID_PATH = prathamjain965_aridzip_path  # Adjust this to your dataset location
test_df, action_classes = parse_arid_splits(ARID_PATH)

print(f"Found {len(test_df)} test videos")
print(f"Action classes: {action_classes}")
print("Sample entries:")
print(test_df.head())


Found 2011 test videos
Action classes: ['Drink', 'Jump', 'Pick', 'Pour', 'Push', 'Run', 'Sit', 'Stand', 'Turn', 'Walk', 'Wave']
Sample entries:
          video_path                                          full_path  \
0   Run/Run_13_1.mp4  /kaggle/input/aridzip/clips_v1.5/Run/Run_13_1.mp4   
1  Run/Run_13_10.mp4  /kaggle/input/aridzip/clips_v1.5/Run/Run_13_10...   
2  Run/Run_13_11.mp4  /kaggle/input/aridzip/clips_v1.5/Run/Run_13_11...   
3  Run/Run_13_12.mp4  /kaggle/input/aridzip/clips_v1.5/Run/Run_13_12...   
4  Run/Run_13_13.mp4  /kaggle/input/aridzip/clips_v1.5/Run/Run_13_13...   

   label action  
0      5    Run  
1      5    Run  
2      5    Run  
3      5    Run  
4      5    Run  


In [29]:
# Create evaluation script
evaluation_code = '''
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

# Video processing
def load_video_frames(video_path, num_frames=16, sample_rate=4):
    """
    Load video frames for MViTv2 input
    """
    cap = cv2.VideoCapture(video_path)
    frames = []

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate frame indices to sample
    total_needed = num_frames * sample_rate
    if frame_count >= total_needed:
        start_idx = (frame_count - total_needed) // 2
        indices = list(range(start_idx, start_idx + total_needed, sample_rate))
    else:
        # Repeat frames if video is too short
        indices = np.linspace(0, frame_count-1, num_frames, dtype=int)

    for i, frame_idx in enumerate(indices):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (224, 224))
            frames.append(frame)

    cap.release()

    # Pad or truncate to exact number of frames needed
    while len(frames) < num_frames:
        frames.append(frames[-1] if frames else np.zeros((224, 224, 3), dtype=np.uint8))
    frames = frames[:num_frames]

    return np.array(frames)

def preprocess_frames(frames):
    """
    Preprocess frames for MViTv2
    """
    # Convert to float and normalize to [0, 1]
    frames = frames.astype(np.float32) / 255.0

    # Normalize with ImageNet stats (commonly used for MViTv2)
    mean = np.array([0.45, 0.45, 0.45])
    std = np.array([0.225, 0.225, 0.225])

    frames = (frames - mean) / std

    # Convert to tensor and rearrange dimensions: T H W C -> C T H W
    frames_tensor = torch.from_numpy(frames).permute(3, 0, 1, 2)

    return frames_tensor

def create_mvitv2_model(num_classes=11, pretrained_path=None):
    """
    Create MViTv2 model for evaluation
    """
    # Using torchvision's MViTv2 as base
    from torchvision.models.video import mvit_v2_s

    # Load base model
    model = mvit_v2_s(weights=None)  # No automatic pretrained weights

    # Modify classifier for ARID dataset (11 classes)
    model.head[-1] = nn.Linear(model.head[-1].in_features, num_classes)

    # Load pretrained weights if provided
    if pretrained_path and os.path.exists(pretrained_path):
        try:
            # Load weights
            checkpoint = torch.load(pretrained_path, map_location='cpu')

            # Handle different checkpoint formats
            if isinstance(checkpoint, dict):
                if 'model_state' in checkpoint:
                    state_dict = checkpoint['model_state']
                elif 'state_dict' in checkpoint:
                    state_dict = checkpoint['state_dict']
                else:
                    state_dict = checkpoint
            else:
                state_dict = checkpoint

            # Load weights with strict=False to ignore classifier layer mismatch
            model.load_state_dict(state_dict, strict=False)
            print(f"Loaded pretrained weights from {pretrained_path}")
        except Exception as e:
            print(f"Warning: Could not load pretrained weights: {e}")
            print("Proceeding with random initialization...")

    return model

def evaluate_arid_dataset(test_df, model, device, action_classes):
    """
    Evaluate MViTv2 on ARID test set
    """
    model.eval()
    all_predictions = []
    all_labels = []
    all_probs = []

    print("Starting evaluation...")

    with torch.no_grad():
        for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Evaluating"):
            try:
                # Load and preprocess video
                frames = load_video_frames(row['full_path'])
                frames_tensor = preprocess_frames(frames)

                # Add batch dimension
                input_tensor = frames_tensor.unsqueeze(0).to(device)

                # Forward pass
                outputs = model(input_tensor)
                probs = F.softmax(outputs, dim=1)
                predicted = torch.argmax(outputs, dim=1)

                all_predictions.append(predicted.cpu().item())
                all_labels.append(row['label'])
                all_probs.append(probs.cpu().numpy()[0])

            except Exception as e:
                print(f"Error processing {row['full_path']}: {e}")
                # Use dummy prediction in case of error
                all_predictions.append(0)
                all_labels.append(row['label'])
                all_probs.append(np.zeros(len(action_classes)))

    return all_predictions, all_labels, np.array(all_probs)

def calculate_metrics(predictions, labels, probs, action_classes):
    """
    Calculate evaluation metrics
    """
    # Top-1 accuracy
    top1_acc = accuracy_score(labels, predictions)

    # Top-5 accuracy
    top5_acc = top_k_accuracy(probs, labels, k=min(5, len(action_classes)))

    # Classification report
    report = classification_report(
        labels, predictions,
        target_names=action_classes,
        digits=4,
        zero_division=0
    )

    # Confusion matrix
    cm = confusion_matrix(labels, predictions)

    return {
        'top1_accuracy': top1_acc,
        'top5_accuracy': top5_acc,
        'classification_report': report,
        'confusion_matrix': cm
    }

def top_k_accuracy(probs, labels, k=5):
    """Calculate top-k accuracy"""
    top_k_preds = np.argsort(probs, axis=1)[:, -k:]
    correct = 0
    for i, label in enumerate(labels):
        if label in top_k_preds[i]:
            correct += 1
    return correct / len(labels)

def plot_results(cm, action_classes, results):
    """
    Plot evaluation results
    """
    # Confusion matrix
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=action_classes,
                yticklabels=action_classes, cmap='Blues')
    plt.title('Confusion Matrix - ARID Dataset Evaluation')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

    # Per-class accuracy
    per_class_acc = []
    for i in range(len(action_classes)):
        if cm[i].sum() > 0:
            acc = cm[i][i] / cm[i].sum()
        else:
            acc = 0
        per_class_acc.append(acc)

    plt.figure(figsize=(12, 6))
    bars = plt.bar(action_classes, per_class_acc)
    plt.title('Per-Class Accuracy - ARID Dataset')
    plt.ylabel('Accuracy')
    plt.xlabel('Action Class')
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)

    # Add value labels on bars
    for bar, acc in zip(bars, per_class_acc):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{acc:.3f}', ha='center', va='bottom')

    plt.tight_layout()
    plt.show()

    print(f"Overall Results:")
    print(f"Top-1 Accuracy: {results['top1_accuracy']:.4f} ({results['top1_accuracy']*100:.2f}%)")
    print(f"Top-5 Accuracy: {results['top5_accuracy']:.4f} ({results['top5_accuracy']*100:.2f}%)")

# Main evaluation function
def main():
    # Load test data (assuming test_df and action_classes are already defined)
    print(f"Evaluating on {len(test_df)} test videos")

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Create model
    pretrained_path = "/content/pretrained_models/mvit_v2_s_kinetics400.pth"
    model = create_mvitv2_model(num_classes=11, pretrained_path=pretrained_path)
    model = model.to(device)

    # Evaluate
    predictions, labels, probs = evaluate_arid_dataset(test_df, model, device, action_classes)

    # Calculate metrics
    results = calculate_metrics(predictions, labels, probs, action_classes)

    # Display results
    print("\\nClassification Report:")
    print(results['classification_report'])

    # Plot results
    plot_results(results['confusion_matrix'], action_classes, results)

    return results

if __name__ == "__main__":
    results = main()
'''

# Save the evaluation script
with open('/content/SlowFast/evaluate_arid_mvitv2.py', 'w') as f:
    f.write(evaluation_code)

print("Created evaluation script")


Created evaluation script


In [31]:
# Execute the evaluation
%cd /content/SlowFast

# Run the evaluation
exec(open('evaluate_arid_mvitv2.py').read())


/content/SlowFast
Evaluating on 2011 test videos
Using device: cpu
Loaded pretrained weights from /content/pretrained_models/mvit_v2_s_kinetics400.pth
Starting evaluation...


Evaluating:   0%|          | 1/2011 [00:01<40:58,  1.22s/it]

Error processing /kaggle/input/aridzip/clips_v1.5/Run/Run_13_1.mp4: Input type (double) and bias type (float) should be the same


Evaluating:   0%|          | 2/2011 [00:02<38:53,  1.16s/it]

Error processing /kaggle/input/aridzip/clips_v1.5/Run/Run_13_10.mp4: Input type (double) and bias type (float) should be the same


...

Error processing /kaggle/input/aridzip/clips_v1.5/Run/Run_13_15.mp4: Input type (double) and bias type (float) should be the same


Evaluating:   0%|          | 8/2011 [00:11<52:54,  1.58s/it]

Error processing /kaggle/input/aridzip/clips_v1.5/Run/Run_13_16.mp4: Input type (double) and bias type (float) should be the same


Evaluating:   0%|          | 8/2011 [00:12<50:52,  1.52s/it]


KeyboardInterrupt: 

In [32]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

In [33]:
def parse_arid_splits(arid_path):
    action_categories = [
        'Drink', 'Jump', 'Pick', 'Pour', 'Push',
        'Run', 'Sit', 'Stand', 'Turn', 'Walk', 'Wave'
    ]
    clips_path = os.path.join(arid_path, "clips_v1.5")
    list_path = os.path.join(arid_path, "list_cvt/split_0")
    test_file = os.path.join(list_path, "split0_test.txt")

    test_data = []
    with open(test_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 3:
                idx_label = int(parts[1])
                vid_rel = parts[2]
                full_path = os.path.join(clips_path, vid_rel)
                if os.path.exists(full_path):
                    test_data.append({
                        'full_path': full_path,
                        'label': idx_label
                    })
    return pd.DataFrame(test_data), action_categories


In [34]:
def load_video_frames(video_path, num_frames=16, sample_rate=4):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    total_needed = num_frames * sample_rate

    if frame_count >= total_needed:
        start = (frame_count - total_needed) // 2
        indices = list(range(start, start + total_needed, sample_rate))
    else:
        indices = np.linspace(0, frame_count - 1, num_frames, dtype=int)

    frames = []
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (224, 224))
        frames.append(frame)
    cap.release()

    # Pad/truncate to num_frames
    while len(frames) < num_frames:
        frames.append(frames[-1].copy())
    return np.stack(frames[:num_frames], axis=0)


In [35]:
def preprocess_frames(frames):
    # Convert to float32 and normalize to [0,1]
    frames = frames.astype(np.float32) / 255.0
    # Use float32 mean/std to avoid dtype mismatch
    mean = np.array([0.45, 0.45, 0.45], dtype=np.float32)
    std  = np.array([0.225, 0.225, 0.225], dtype=np.float32)
    frames = (frames - mean) / std
    # NHWC -> NCTHW
    tensor = torch.from_numpy(frames).permute(3, 0, 1, 2)
    return tensor

In [41]:
def create_mvitv2_model(num_classes=11, pretrained_path=None):
    from torchvision.models.video import mvit_v2_s
    model = mvit_v2_s(weights=None)
    # Load pretrained weights if available
    if pretrained_path and os.path.exists(pretrained_path):
        ckpt = torch.load(pretrained_path, map_location='cpu')
        state = ckpt.get('model_state', ckpt.get('state_dict', ckpt))
        model.load_state_dict(state, strict=False)
        print(f"Loaded pretrained weights from {pretrained_path}")
    return model

In [51]:
test_df

,full_path,label
0,/kaggle/input/aridzip/clips_v1.5/Run/Run_13_1.mp4,5
1,/kaggle/input/aridzip/clips_v1.5/Run/Run_13_10...,5
2,/kaggle/input/aridzip/clips_v1.5/Run/Run_13_11...,5
3,/kaggle/input/aridzip/clips_v1.5/Run/Run_13_12...,5
4,/kaggle/input/aridzip/clips_v1.5/Run/Run_13_13...,5
...,...,...
2006,/kaggle/input/aridzip/clips_v1.5/Push/Push_6_5...,4
2007,/kaggle/input/aridzip/clips_v1.5/Push/Push_6_6...,4
2008,/kaggle/input/aridzip/clips_v1.5/Push/Push_6_7...,4
2009,/kaggle/input/aridzip/clips_v1.5/Push/Push_6_8...,4


In [52]:
action_categories = [
        'Drink', 'Jump', 'Pick', 'Pour', 'Push',
        'Run', 'Sit', 'Stand', 'Turn', 'Walk', 'Wave'
    ]

In [55]:
def generate_predictions(test_df, model, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Predicting"):
            try:
                frames = load_video_frames(row['full_path'])
                tensor = preprocess_frames(frames).unsqueeze(0).to(device)
                outputs = model(tensor)
                pred = torch.argmax(outputs, dim=1).item()
            except Exception as e:
                # Fallback to class 0 if an error occurs
                pred = 0
            preds.append(pred)
            print(f"Actual : {action_categories[row['label']]} , Preds : {preds[-1]}")
    return preds


In [39]:
ARID_PATH = "/kaggle/input/aridzip"
PTH_PATH  = "/content/pretrained_models/mvit_v2_s_kinetics400.pth"

In [40]:
test_df, action_classes = parse_arid_splits(ARID_PATH)
print(f"Total test videos: {len(test_df)}")

Total test videos: 2011


In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model = create_mvitv2_model(num_classes=len(action_classes),
                            pretrained_path=PTH_PATH)
model.to(device)

Using device: cpu
Loaded pretrained weights from /content/pretrained_models/mvit_v2_s_kinetics400.pth


MViT(
  (conv_proj): Conv3d(3, 96, kernel_size=(3, 7, 7), stride=(2, 4, 4), padding=(1, 3, 3))
  (pos_encoding): PositionalEncoding()
  (blocks): ModuleList(
    (0): MultiscaleBlock(
      (norm1): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
      (norm2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
      (attn): MultiscaleAttention(
        (qkv): Linear(in_features=96, out_features=288, bias=True)
        (project): Sequential(
          (0): Linear(in_features=96, out_features=96, bias=True)
        )
        (pool_q): Pool(
          (pool): Conv3d(96, 96, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), groups=96, bias=False)
          (norm_act): Sequential(
            (0): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          )
        )
        (pool_k): Pool(
          (pool): Conv3d(96, 96, kernel_size=(3, 3, 3), stride=(1, 8, 8), padding=(1, 1, 1), groups=96, bias=False)
          (norm_act): Sequential(
            (0): LayerNorm((96,

In [56]:
raw_labels = generate_predictions(test_df, model, device)
print("Raw predicted labels:", raw_labels)

Predicting:   0%|          | 1/2011 [00:06<3:21:26,  6.01s/it]

Actual : Run , Preds : 61


Predicting:   0%|          | 2/2011 [00:10<3:00:12,  5.38s/it]

Actual : Run , Preds : 61


Predicting:   0%|          | 3/2011 [00:16<3:08:34,  5.63s/it]

...

Actual : Run , Preds : 61


Predicting:   2%|▏         | 36/2011 [03:20<3:02:10,  5.53s/it]

Actual : Run , Preds : 61


Predicting:   2%|▏         | 36/2011 [03:20<3:03:33,  5.58s/it]


KeyboardInterrupt: 

In [44]:
raw_labels

NameError: name 'raw_labels' is not defined